In [1]:
# import libraries

In [ ]:
import time
import numpy
import pandas
import sklearn.preprocessing
import sklearn.model_selection

In [2]:
# import data as csv using relative path
df_data = pandas.read_csv( "u.data", delimiter = "\t", header = None )

In [3]:
# define dataframe's column names
df_data.columns = ["userid", "movieid", "rating", "timestamp"]

In [4]:
# drop timestamp column
df_data.drop("timestamp", inplace = True, axis = 1)

In [5]:
# sort dataframe by user id to index it later
df_data_sorted_by_user = df_data.sort_values(by=['userid'])

In [6]:
df_data_sorted_by_user

,userid,movieid,rating
66567,1,55,5
62820,1,203,4
10207,1,183,5
9971,1,150,5
22496,1,68,4
...,...,...,...
96823,943,427,4
70902,943,12,5
84518,943,284,2
72321,943,62,3


In [7]:
# multi-index dataframe using userid and movieid
df_data_indexed_by_user = df_data_sorted_by_user.set_index(['userid', 'movieid'])

In [8]:
df_data_indexed_by_user

rating
userid movieid        
1      55            5
       203           4
       183           5
       150           5
       68            4
...                ...
943    427           4
       12            5
       284           2
       62            3
       210           4

[100000 rows x 1 columns]

In [9]:
# find all users ratings' mean
df_mean_ratings = df_data_indexed_by_user.mean(level='userid')

In [10]:
df_mean_ratings

,rating
userid,
1,3.610294
2,3.709677
3,2.796296
4,4.333333
5,2.874286
...,...
939,4.265306
940,3.457944
941,4.045455


In [11]:
# define a numpy array that represents all indices
np_indexes = numpy.array(numpy.meshgrid(numpy.arange(1, df_mean_ratings.size + 1), numpy.arange(1, df_data_indexed_by_user.groupby('movieid').size().index.size + 1))).T.reshape(-1,2)

In [12]:
# convert array to tuple
tuples = tuple(map(tuple, np_indexes))

In [13]:
# pass tuple to pandas library
indexes = pandas.MultiIndex.from_tuples(tuples, names=['userid', 'movieid'])

In [14]:
# create a pandas series using the computed indices and filling all cells with zeros (sub-part c of project)
s_data = pandas.Series(0, index=indexes)

In [15]:
s_data

userid  movieid
1       1          0
        2          0
        3          0
        4          0
        5          0
                  ..
943     1678       0
        1679       0
        1680       0
        1681       0
        1682       0
Length: 1586126, dtype: int64

In [16]:
# convert series to dataframe
df_zeros = s_data.to_frame().rename(columns={0: "rating"})

In [17]:
# merge the dataframe filled with zeros and the dataframe with the user-movie ratings
df_data_merged = pandas.merge(df_zeros, df_data_indexed_by_user, how='left', on=['userid', 'movieid'])

In [18]:
# replace zeros with user-movie ratings if rating is defined and rename the column  
df_data_merged.drop("rating_x", inplace = True, axis = 1)
df_complete = df_data_merged.fillna(0)
df_complete = df_complete.rename(columns={"rating_y": "rating"})

In [22]:
# initialize numpy array with users' mean movie ratings
np_zeros = numpy.empty([df_mean_ratings.size + 1, df_data_indexed_by_user.groupby('movieid').size().index.size], dtype=float)
for i in numpy.arange(1, df_mean_ratings.size + 1):
    np_zeros[i] = numpy.repeat(float(df_mean_ratings.loc[i]), 1682)
np_mean_ratings = numpy.delete(np_zeros, 0, 0)

In [24]:
np_mean_ratings

array([[3.61029412, 3.61029412, 3.61029412, ..., 3.61029412, 3.61029412,
        3.61029412],
       [3.70967742, 3.70967742, 3.70967742, ..., 3.70967742, 3.70967742,
        3.70967742],
       [2.7962963 , 2.7962963 , 2.7962963 , ..., 2.7962963 , 2.7962963 ,
        2.7962963 ],
       ...,
       [4.04545455, 4.04545455, 4.04545455, ..., 4.04545455, 4.04545455,
        4.04545455],
       [4.26582278, 4.26582278, 4.26582278, ..., 4.26582278, 4.26582278,
        4.26582278],
       [3.41071429, 3.41071429, 3.41071429, ..., 3.41071429, 3.41071429,
        3.41071429]])

In [25]:
# use the previous array to centralize data
start = time.time()
for i,j in df_complete.index:
    if df_complete.loc[(i,j), 'rating'] != 0:
        df_complete.loc[i,j] = df_complete.loc[(i,j), 'rating'] - np_mean_ratings[i - 1][j - 1]
end = time.time()
print(end - start)

298.839861869812


In [26]:
# normalize the new dataframe
ratings = df_complete.values
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
ratings_scaled = min_max_scaler.fit_transform(ratings)
df_complete.loc[:, ('rating')] = ratings_scaled

In [27]:
# save the dataframe
# !customize the path below!
df_complete.to_csv('/home/andreas/data/complete_normalized_data.csv', encoding='utf-8')

In [28]:
# create a tabular view of the data 
df_tabular = df_complete.reset_index().pivot(index='userid', columns='movieid', values='rating')

In [29]:
df_tabular

movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,0.707095,0.430554,0.568825,0.430554,0.430554,0.707095,0.568825,0.154012,0.707095,0.430554,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
2,0.555083,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.278541,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
3,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
4,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
5,0.670593,0.532322,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.616526,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
940,0.514940,0.514940,0.514940,0.313349,0.514940,0.514940,0.589890,0.728161,0.451619,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
941,0.646925,0.514940,0.514940,0.514940,0.514940,0.514940,0.508655,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
